In [48]:
!pip uninstall rulprediction --yes

Found existing installation: rulprediction 1.0
Uninstalling rulprediction-1.0:
  Successfully uninstalled rulprediction-1.0


In [49]:
!pip install git+https://github.com/sashaatlasov/RUL-Prediction.git

  Cloning https://github.com/sashaatlasov/RUL-Prediction.git to /private/var/folders/cr/12mnwryn4bj3dhrct6_xvw340000gn/T/pip-req-build-hkpz9ef2
  Running command git clone --filter=blob:none --quiet https://github.com/sashaatlasov/RUL-Prediction.git /private/var/folders/cr/12mnwryn4bj3dhrct6_xvw340000gn/T/pip-req-build-hkpz9ef2
  Resolved https://github.com/sashaatlasov/RUL-Prediction.git to commit 196d10f983a1ab058fd16b4c02a241183a2d9346
  Preparing metadata (setup.py) ... done
  Created wheel for rulprediction: filename=rulprediction-1.0-py3-none-any.whl size=10123 sha256=5da5bf4a1aa233df89a82a35d312d894e6a9ab6bda7a2d7917d4d5bc58bcc227
  Stored in directory: /private/var/folders/cr/12mnwryn4bj3dhrct6_xvw340000gn/T/pip-ephem-wheel-cache-42vj73pz/wheels/d9/26/2f/4a379e714991d395b358c8aebe5b708c26e61c28df8e5e8036
Successfully built rulprediction


In [10]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

In [11]:
class CMAPSS(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        target = torch.tensor(self.y[idx], dtype=torch.float32)
        sensors = torch.tensor(self.X[idx], dtype=torch.float32)
        return target, sensors.squeeze(-1)

In [12]:
def get_data(k):
    X_train = sio.loadmat(f'data/F00{k}/trainX.mat')['trainX']
    X_test = sio.loadmat(f'data/F00{k}/testX.mat')['testX']

    y_train = sio.loadmat(f'data/F00{k}/trainY.mat')['trainY'].T
    y_test = sio.loadmat(f'data/F00{k}/testY.mat')['testY'].T
    
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2, random_state=1)
    
    train_data = CMAPSS(X_train, y_train) 
    val_data = CMAPSS(X_val, y_val) 
    test_data = CMAPSS(X_test, y_test) 
                        
    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=256, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=256)
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=256)
                        
    return train_loader, val_loader, test_loader

In [13]:
from rulpred.RULprediction import RULpredictor
from rulpred.training import *

from hparams import config

def main(k, device='cpu', num_epochs=100):
    if k % 2 == 0:
        win_size = 60
    else:
        win_size = 40
        
    model = RULpredictor(win_size)
    model.to(device)

    optim = torch.optim.Adam(model.parameters(), lr=config["learning_rate_start"])
    train_dataloader, val_dataloader, test_dataloader = get_data(k)

    for i in range(1, 1 + 1):
        loss = train_epoch(model, train_dataloader, optim, device=device)
        if i % 5 == 0:
            val_loss = validate(model, val_dataloader, device)
    test_loss = validate(model, test_dataloader, device)
    torch.save(model, f'Model_F00{k}.pt')

In [14]:
main(1)

  0%|          | 0/1 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.8/site-packages/rulpred/training.py:34: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([100, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(mean * model.rul_max, t * model.rul_max, reduction='none')
100%|██████████| 1/1 [00:02<00:00,  2.81s/it]


In [15]:
model = torch.load('Model_F001.pt')

In [16]:
dast = torch.load('DAST_F001.pth')

In [17]:
from rulpred.evaluation.plot_prediction import plot_traj

In [21]:
data = sio.loadmat('plot_testX.mat')
X_test = data['testX']
y_test = data['testY'].T
masks = data['mask'].T

In [23]:
plot_traj(24, model, dast, X_test, y_test, masks)

NameError: name 'masks' is not defined